# Backend API Prototyping and A2A Communication

---

##  Final System Prompt for `06_backend_api_prototyping.ipynb`

###  Notebook Title:
**TinyTutor Capstone Notebook 06: Backend API Prototyping and A2A Communication**

###  Objective:
Simulate the **production deployment architecture** of TinyTutor using the **Vertex AI Agent Engine**. This notebook must demonstrate:
- Packaging the modular agent system into a deployable backend
- Implementing **Agent-to-Agent (A2A) communication**
- Simulating **Opal integration** as a downstream consumer
- Generating deployment and cleanup commands for cloud execution

---

###  System Prompt:
> Generate runnable Python code for `06_backend_api_prototyping.ipynb` that simulates the production deployment of TinyTutor. Implement the following:
>
> 1. **Setup**:
>     - Include ADK and Gemini imports
>     - Simulate GCP project and location configuration
>
> 2. **A2A Simulation**:
>     - Define a `ReviewerAgent` (simulating `backend/app/agents/reviewer_agent.py`) with instructions for quality checking
>     - Expose it using `to_a2a()` to simulate remote service deployment
>     - Define a `ScriptAgent` that delegates to `ReviewerAgent` using `RemoteA2aAgent` or `AgentTool`
>
> 3. **Opal Integration (Optional)**:
>     - Define a `generate_opal_prototype(lesson_topic: str) -> str` FunctionTool
>     - Simulate how an Opal-generated parent/teacher interface might consume the A2A-exposed TinyTutor agent
>
> 4. **Backend Packaging Simulation**:
>     - Simulate creating a `backend/` directory with:
>         - `requirements.txt` listing ADK and dependencies
>         - `app/main.py` as the entry point
>         - `app/pipelines/tutoring_pipeline.py` as the orchestrator
>
> 5. **Deployment Command Generation**:
>     - Generate the `adk deploy` CLI command with placeholders for:
>         - `--project_id`
>         - `--location`
>         - `--entry_point`
>         - `--root_dir`
>
> 6. **Post-Deployment Test Simulation**:
>     - Simulate a call to the deployed agent using `vertexai.agent_engines.run()` with a sample query
>     - Confirm that the A2A delegation chain works
>
> 7. **Cleanup Command**:
>     - Provide the `adk delete` CLI command to remove the deployed agent engine and manage cloud costs
>
> 8. **Best Practices**:
>     - Use structured logging and comments
>     - Mention versioning and rollback strategies
>     - Explain how this setup supports CI/CD and modular deployment

---

##  Final Checklist for `06_backend_api_prototyping.ipynb`

| **Category**         | **Requirement**                                                                                                                                       | **Source/Justification**                                                                 |
|----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| **Core Concept**      | Prototype-to-Production transition with A2A interoperability                                                                                         | Capstone deployment requirement                                                           |
| **Goal**              | Simulate backend packaging and agent delegation using A2A protocol                                                                                   | Demonstrates modularity and service boundaries                                            |
| **Dependencies**      | Requires agents and tools from Notebooks 01–05                                                                                                       | Ensures full system integration                                                           |
| **Required Tools**    | - `to_a2a()` <br> - `RemoteA2aAgent` <br> - `adk deploy` and `adk delete` CLI commands                                                               | Enables cloud deployment and remote agent consumption                                     |
| **Architecture**      | - Microservices pattern <br> - Agent Card exposure <br> - Modular backend directory                                                                  | Aligns with Vertex AI Agent Engine deployment model                                       |
| **Opal Integration**  | - Define `generate_opal_prototype()` tool <br> - Simulate downstream consumption of TinyTutor via A2A                                                 | Supports bonus Capstone criteria for interface integration                                |
| **Execution**         | - Simulate deployment and post-deployment test <br> - Confirm A2A delegation works                                                                  | Validates production readiness                                                            |
| **Good Practices**    | - CI/CD simulation with `requirements.txt`, `main.py`, `tutoring_pipeline.py` <br> - Versioning and rollback awareness <br> - Cost management cleanup | Ensures maintainability, scalability, and budget control                                  |
| **Documentation**     | - Inline comments <br> - Markdown explanations                                                                                                       | Supports Capstone reviewers and future collaborators                                      |

---

###  What We’ll Have When This Code Is Done

-  A simulated backend structure ready for deployment
-  A working A2A delegation chain between agents
-  CLI commands for deploying and cleaning up on Vertex AI Agent Engine
-  A mock Opal integration point for parent/teacher-facing interfaces
-  A post-deployment test simulation to validate the system
-  Clear documentation and inline logic to support Capstone delivery and DevOps alignment

---



#  TinyTutor Capstone Notebook 06: Backend API Prototyping and A2A Communication

This notebook simulates the production deployment of TinyTutor using mock classes. It demonstrates:
- Packaging the agent system into a modular backend structure
- Agent-to-Agent (A2A) communication across service boundaries
- Simulated deployment and post-deployment interaction
This fulfills the Capstone requirement for production readiness and modular architecture.

In [40]:
# import logging
# logging.basicConfig(level=logging.INFO)

In [41]:
from typing import Callable, Dict, Any

# Simulated A2A exposure and consumption
class RemoteA2aAgent:
    def __init__(self, name: str, endpoint: str):
        self.name = name
        self.endpoint = endpoint

    def call(self, input_text: str) -> str:
        print(f"[RemoteA2aAgent] Calling {self.name} at {self.endpoint} with input: {input_text}")
        return f"{self.name} response to: {input_text}"

class LlmAgent:
    def __init__(self, name: str, system_instruction: str, tools: list = None):
        self.name = name
        self.system_instruction = system_instruction
        self.tools = tools or []

    def run(self, input_text: str) -> Dict[str, Any]:
        print(f"\n[{self.name}] Instruction: {self.system_instruction}")
        print(f"[{self.name}] Input: {input_text}")
        result = {}
        for tool in self.tools:
            result[tool.name] = tool.call(input_text)
        return result

class FunctionTool:
    def __init__(self, name: str, function: Callable):
        self.name = name
        self.function = function

    def call(self, input_text: str) -> str:
        return self.function(input_text)

##  Step 1: Define ReviewerAgent and ScriptAgent

ReviewerAgent is exposed as a remote service. ScriptAgent delegates to it using A2A.

In [42]:
# ReviewerAgent (simulated remote service)
def review_script(input_text: str) -> str:
    return f"Review passed for: {input_text}"

reviewer_tool = FunctionTool(name="review_script", function=review_script)

reviewer_agent = LlmAgent(
    name="ReviewerAgent",
    system_instruction="Review the script for quality and safety.",
    tools=[reviewer_tool]
)

# Expose ReviewerAgent as A2A
reviewer_endpoint = "https://vertex.fake.endpoint/reviewer"
remote_reviewer = RemoteA2aAgent(name="ReviewerAgent", endpoint=reviewer_endpoint)

# ScriptAgent delegates to ReviewerAgent
class ScriptAgent:
    def __init__(self, remote_agent: RemoteA2aAgent):
        self.remote_agent = remote_agent

    def run(self, topic: str) -> Dict[str, str]:
        script = f"Story about: {topic}"
        review = self.remote_agent.call(script)
        return {
            "script": script,
            "review": review
        }

script_agent = ScriptAgent(remote_agent=remote_reviewer)

In [43]:
def generate_opal_prototype(lesson_topic: str) -> str:
    return f" Opal UI: Displaying lesson preview for topic: '{lesson_topic}'"

opal_tool = FunctionTool(name="generate_opal_prototype", function=generate_opal_prototype)
print(opal_tool.call("Why do stars twinkle?"))

 Opal UI: Displaying lesson preview for topic: 'Why do stars twinkle?'


##  Step 2: Simulate Backend Packaging

Create a mock backend file structure for deployment.

In [44]:
import os

os.makedirs("backend/app/pipelines", exist_ok=True)

with open("backend/requirements.txt", "w") as f:
    f.write("google-cloud-aiplatform\n")

with open("backend/app/main.py", "w") as f:
    f.write("# Entry point for TinyTutor\n")

with open("backend/app/pipelines/tutoring_pipeline.py", "w") as f:
    f.write("# Orchestrator logic for TinyTutor pipeline\n")

print("Simulated backend structure created.")

Simulated backend structure created.


##  Step 3: Generate Deployment Commands

Use ADK CLI syntax to deploy to Vertex AI Agent Engine.

These commands simulate how you would deploy the backend using the ADK CLI (if Agent Builder were available).

In [45]:
project_id = "tinytutor-capstone"
location = "us-central1"

deploy_command = f"adk deploy --project_id={project_id} --location={location} --entry_point=app/main.py --root_dir=backend"
delete_command = f"adk delete --project_id={project_id} --location={location} --entry_point=app/main.py"

print(" Deployment Command:\n", deploy_command)
print("\n Cleanup Command:\n", delete_command)

 Deployment Command:
 adk deploy --project_id=tinytutor-capstone --location=us-central1 --entry_point=app/main.py --root_dir=backend

 Cleanup Command:
 adk delete --project_id=tinytutor-capstone --location=us-central1 --entry_point=app/main.py


##  Step 4: Simulate Post-Deployment Interaction

Use Vertex SDK-style call to test the deployed agent.

This block simulates how you would call the deployed agent using the Vertex SDK if deployed via Agent Builder.

In [46]:
def vertexai_agent_engines_run(agent_name: str, input_text: str) -> str:
    print(f"[Vertex SDK] Running agent '{agent_name}' with input: {input_text}")
    return f"Response from {agent_name}: Lesson generated for '{input_text}'"

response = vertexai_agent_engines_run("TinyTutorCoordinator", "Explain gravity like I'm 5")
print("\n Post-Deployment Response:\n", response)

[Vertex SDK] Running agent 'TinyTutorCoordinator' with input: Explain gravity like I'm 5

 Post-Deployment Response:
 Response from TinyTutorCoordinator: Lesson generated for 'Explain gravity like I'm 5'


##  Fallback Deployment: Gemini + Vertex AI SDK

This section simulates the TinyTutor backend using Gemini via the Vertex AI SDK, without requiring Agent Builder access.

In [47]:
!pip install -q google-cloud-aiplatform

In [48]:
try:
    from google.colab import auth
    auth.authenticate_user()
except:
    pass

In [49]:
import vertexai

# Initialize Vertex AI
vertexai.init(project="tinytutor-capstone", location="us-central1")

# Load Gemini model
from vertexai.language_models import ChatModel
# chat_model = ChatModel.from_pretrained("gemini-pro")
# chat_model = ChatModel.from_pretrained("gemini-1.5-flash-preview")
# chat_model = ChatModel.from_pretrained("gemini-1.5-pro-preview")
chat_model = ChatModel.from_pretrained("chat-bison")
chat = chat_model.start_chat()

# Step 1: Simplify the topic
topic = "Explain the Theory of Relativity like I'm 5 using dinosaur characters"
response_1 = chat.send_message(topic)
eli5_explanation = response_1.text

# Step 2: Turn into a story
story_prompt = f"Turn this into a short story for kids with characters: {eli5_explanation}"
response_2 = chat.send_message(story_prompt)
final_script = response_2.text

# Simulated multimodal outputs
audio_uri = "data/audio/dino_voice.mp3"
video_uri = "data/video/dino_scene.mp4"

# Display results
print(" ELI5 Explanation:\n", eli5_explanation)
print("\n Final Story Script:\n", final_script)
print("\n Audio URI:", audio_uri)
print(" Video URI:", video_uri)
print("\n Fallback Execution Complete")

NotFound: 404 Publisher Model `projects/tinytutor-capstone/locations/us-central1/publishers/google/models/chat-bison@002` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions